In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install fake_useragent html2text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for html2text: filename=html2text-2024.2.26-py3-none-any.whl size=33111 sha256=970cb6d6a7338691f933374f65478ed1cbbccfb1507347f80d001be8cca644c4
  Stored in directory: /root/.cache/pip/wheels/f3/96/6d/a7eba8f80d31cbd188a2787b81514d82fc5ae6943c44777659
Successfully built html2text


In [ ]:
import warnings

warnings.simplefilter("ignore")

In [ ]:
from urllib.request import urlopen, Request

In [ ]:
from bs4 import BeautifulSoup
import requests
import time
import concurrent.futures
import csv
from fake_useragent import UserAgent
from tqdm import tqdm
import html2text

In [ ]:
ua = UserAgent()

In [ ]:
nairaland_domain="https://www.nairaland.com"

In [ ]:
def get_posts_link_for_page_urllib(page_link,agent):
  req = Request(url=page_link,headers={'user-agent': agent})
  response = urlopen(req)
  soup_page = BeautifulSoup(response,"html.parser")
  try:
    posts_info=soup_page.find("div",class_="body").find_all("table")[2].tr.th.find_all("td")
  except AttributeError:
    posts_info=soup_page.find("div",class_="body").find_all("table")[1].tr.th.find_all("td")
  nairaland_domain="https://www.nairaland.com"
  links=[nairaland_domain+i.find("b").a.get("href") for i in posts_info]
  return links

In [ ]:
def get_posts_links_for_page_eror(page_link,agent):
  page_req=requests.get(page_link,headers={'User-Agent':agent}).text
  soup_page=BeautifulSoup(page_req,"html.parser")
  #print(soup_page.find("div",class_="body").find_all("tr")[1].find_all("td")[0].find_all("td",class_="w")[0].prettify())#)#)
  posts_info=soup_page.find("div",class_="body").find_all("table")[2].tr.th.find_all("tr")

  nairaland_domain="https://www.nairaland.com"
  links=[nairaland_domain+i.find("b").a.get("href") for i in posts_info]
  return links

def get_posts_links_for_page(page_link,agent=ua.chrome):
    """This takes in a link of a page under each nairaland section and returrns a list of links to each post in that page"""
    try:
      return get_posts_links_for_page_eror(page_link,agent)
    except:
      try:
        return get_posts_links_for_page_eror(page_link,ua.safari)
      except:
        try:
          return get_posts_links_for_page_eror(post_link,agent=ua.firefox)
        except:
          try:
            return get_posts_links_for_page_eror(post_link,agent=ua.edge)
          except:
            try:
              return get_posts_links_for_page_eror(post_link,agent=ua.random)
            except:
              try:
                return get_posts_links_for_page_eror(post_link,agent="*")
              except:
                try:
                  return get_posts_link_for_page_urllib(page_link,agent=ua.random)
                except:
                  return []

In [ ]:
get_posts_links_for_page("https://www.nairaland.com/travel")

['https://www.nairaland.com/7767052/uk-student-visa-tier-4',
 'https://www.nairaland.com/7194807/general-usa-student-visa-enquiries-part',
 'https://www.nairaland.com/6969367/general-uk-visa-enquiries-part',
 'https://www.nairaland.com/7432492/living-usa-life-immigrant-part',
 'https://www.nairaland.com/8098253/ultimate-ujjain-omkareshwar-tour-package',
 'https://www.nairaland.com/7776621/canadian-student-visa-thread-part',
 'https://www.nairaland.com/8097134/told-need-millions-japa',
 'https://www.nairaland.com/4672912/all-tef-candidates-canada-immigration',
 'https://www.nairaland.com/8097942/british-aviation-reports-air-peace',
 'https://www.nairaland.com/8098192/official-work-permits-canada-how',
 'https://www.nairaland.com/8098219/ticket-price-slash-flight-south',
 'https://www.nairaland.com/7788707/memoirs-chile',
 'https://www.nairaland.com/944102/general-south-africa-visa-enquiries',
 'https://www.nairaland.com/8098206/directions-please',
 'https://www.nairaland.com/8098144/top

In [ ]:
def clean_likes(likes):
  if " Likes" in likes:
    likes=likes.replace(" Likes","")
  else:
    likes=likes.replace(" Like","")
  likes.replace(" ","")
  try:
    return int(likes)
  except ValueError:
    return 0

In [ ]:
def get_title_and_post_content_error(post_link,agent=ua.chrome):
  req=requests.get(post_link,headers={'User-Agent':agent}).text
  soup=BeautifulSoup(req,"html.parser")
  body=soup.find("div",class_="body")
  topic_h2=body.find("h2").text
  topic_split=topic_h2.split(" - ")
  post_header=" - ".join(topic_split[:len(topic_split)-2])
  post_content=body.find("div",class_="narrow").text
  posts=body.find_all("table",summary="posts")[0].find_all("td",class_="l w pd")
  best_response=None
  max_likes=0
  for post in posts[1:]:
    post_reply=post.find("div",class_="narrow").text
    try:
      post_likes=clean_likes(post.find('p',class_="s").b.text)#int(.replace(" Likes",""))
    except:
      post_likes=0
    if post_likes>=max_likes:
      best_response=post_reply
      max_likes=post_likes
  #print(best_response)
  return {"title":post_header,"content":html2text.html2text(str(post_content)),"response":html2text.html2text(str(best_response)),"likes":max_likes}

def get_title_and_post_content_urllib(post_link,agent=ua.chrome):
  req = Request(url=post_link,headers={'user-agent': agent})
  response = urlopen(req)
  soup = BeautifulSoup(response,"html.parser")
  body=soup.find("div",class_="body")
  topic_h2=body.find("h2").text
  topic_split=topic_h2.split(" - ")
  post_header=" - ".join(topic_split[:len(topic_split)-2])
  post_content=html2text.html2text(str(body.find("div",class_="narrow")))
  posts=body.find_all("table",summary="posts")[0].find_all("td",class_="l w pd")
  best_response=None
  max_likes=0
  for post in posts[1:]:
    post_reply=post.find("div",class_="narrow").text
    try:
      post_likes=clean_likes(post.find('p',class_="s").b.text)#int(.replace(" Likes",""))
    except:
      post_likes=0
    if post_likes>=max_likes:
      best_response=post_reply
      max_likes=post_likes
  #print(best_response)
  return {"title":post_header,"content":html2text.html2text(str(post_content)),"response":html2text.html2text(str(best_response)),"likes":max_likes}

def get_title_and_post_content(post_link,agent=ua.chrome):
  """
      Takes in a link to a post and return the title and the original poster's post
  """
  try:
    return get_title_and_post_content_error(post_link,agent=agent)

  except:
    try:
      return get_title_and_post_content_error(post_link,agent=ua.safari)
    except:
      try:
        return get_title_and_post_content_error(post_link,agent=ua.firefox)
      except:
        try:
          return get_title_and_post_content_error(post_link,agent=ua.edge)
        except:
          try:
            return get_title_and_post_content_error(post_link,agent=ua.random)
          except:
              try:
                return get_title_and_post_content_error(post_link,agent="*")
              except:
                try:
                  return get_title_and_post_content_urllib(post_link,agent=ua.random)
                except :
                  #return {"content":""}
                  return {"title":"","content":"","response":"","likes":""}

In [ ]:
#get the sections
req=requests.get("https://www.nairaland.com/home",headers={'User-Agent':ua.random}).text
soup=BeautifulSoup(req,"html.parser")
g=soup.find("td",class_="l").find_all("a")

dict_={a.b.text:nairaland_domain+a.get("href") for a in g}
for sec in ["Nairaland / General","Entertainment","Science/Technology"]:
    dict_.pop(sec)

In [ ]:
dict_["TV-Movies"]=dict_["TV/Movies"]
dict_['Music-Radio']=dict_['Music/Radio']

In [ ]:
def get_section_links(section_name,dict=dict_):
  number_of_pages=1000#number_of_pages_for_each_section
  #gets the links to all the pages in each section
  return [dict_[section_name]+f"/{page_no}" for page_no in range(number_of_pages)]

In [ ]:
def write_single_post_info_to_csv(post_link):
  #writes all the REQUIRED info from a page to a csv given the page link
  post_info=get_title_and_post_content(post_link) #(post_info)
  #print(post_info)
  list_texts.append(post_info)

In [ ]:
def write_info_from_section_page_csv(section_link):
  """takes in a single page from a section  and performs the 'write_single_post_info_to_csv' on all the posts in the shown in the page"""
  MAX_THREADS=30
  posts_links=get_posts_links_for_page(section_link) #all the link in that page in the
  #print(posts_links)
  threads =35
  if posts_links:
    with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
      executor.map(write_single_post_info_to_csv,posts_links)
  else:
    pass
  print(section_link)

#### Just Insert the name of the section you want to download data from and the data will be webscraped. The datasets from various section will be combined in a later notebook

In [ ]:
import pandas as pd

In [ ]:
#csv_file=open(path,"a")
#csv_writer=csv.writer(csv_file)
list_texts=[]
start=time.time()
section="Religion"
path=f"/content/drive/MyDrive/Nairaland_data/data_{section}.csv"
section_links=get_section_links(section)
threads = 100
with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
  executor.map(write_info_from_section_page_csv,section_links)
end=time.time()
print(f"It took {end-start} seconds to run the code")

https://www.nairaland.com/religion/1030
https://www.nairaland.com/religion/1028
https://www.nairaland.com/religion/1027
https://www.nairaland.com/religion/1024
https://www.nairaland.com/religion/1005
https://www.nairaland.com/religion/1026
https://www.nairaland.com/religion/1029
https://www.nairaland.com/religion/1017
https://www.nairaland.com/religion/1021
https://www.nairaland.com/religion/1032
https://www.nairaland.com/religion/1033
https://www.nairaland.com/religion/1019
https://www.nairaland.com/religion/1037
https://www.nairaland.com/religion/1047
https://www.nairaland.com/religion/1042
https://www.nairaland.com/religion/1040
https://www.nairaland.com/religion/1048
https://www.nairaland.com/religion/1039
https://www.nairaland.com/religion/1044
https://www.nairaland.com/religion/1043
https://www.nairaland.com/religion/1036
https://www.nairaland.com/religion/1041
https://www.nairaland.com/religion/1022
https://www.nairaland.com/religion/1000
https://www.nairaland.com/religion/1018


In [ ]:
pd.DataFrame(list_texts).to_csv(path,escapechar='\\')

In [ ]:
path

In [ ]:
df=pd.read_csv(path)

In [ ]:
df

,Unnamed: 0,title,content,response,likes
0,0,Biafra Pastor Predicts That Covid-20 And Covid...,Nigeria & Biafra Pastor predicts that Covid-20...,Image123:i thought he threatened Coronavirus t...,1.0
1,1,The Wages Of Sin Is Still Death,"Though many people seem to have forgotten, I w...",None\n\n,0.0
2,2,Interpretation Of The Dream Of Seeing Dead Peo...,Listen to Hear the interpretation of the dream...,Severe malaria fever.\n\n,0.0
3,3,The Living Word Of God; Christ Jesus Crucified...,Only the living word of God; Christ Jesus Cruc...,None\n\n,0.0
4,4,"Pastors Or Cult Leaders?James Macdonald, Andy ...",James MacDonald is currently under investigati...,None\n\n,0.0
...,...,...,...,...,...
69013,69013,NaN,NaN,NaN,NaN
69014,69014,The Final Lie,"""Surely, I come quickly."" (Revelation 22:20)Th...","1 John 2:18-19 ""Children, it is the last hour;...",0.0
69015,69015,NaN,NaN,NaN,NaN
69016,69016,NaN,NaN,NaN,NaN


In [ ]:

list_texts

[{'title': 'Biafra Pastor Predicts That Covid-20 And Covid-21 Are Coming',
  'content': 'Nigeria & Biafra Pastor predicts that Covid-20 and Covid-21 are coming.\nhttps://www.youtube.com/watch?v=R2dWQHz7z0A\n\n',
  'response': 'Image123:i thought he threatened Coronavirus the other time with who born you,\nwho you? It is unfortunate that any human being thinks or says this\nbusinessman is a man of God too.How come no one predicted the 3 month lockdown\nis what I find interesting to be. I suppose with Covid-20 and Covid-21\nprediction, we should be looking forward to a 6 months and 12 months lockdown\nnow\n\n',
  'likes': 1},
 {'title': 'The Wages Of Sin Is Still Death',
  'content': 'Though many people seem to have forgotten, I want to remind you today that the\nwages of sin is still death. Death is a salary for sinners as stated in Romans\n6:23. The sinner cannot dodge it; you work and you must get paid. Ezekiel\n18:20 says:“The soul that sinneth, it shall die. The son shall not bear t

In [ ]:
df["response"].value_counts()

response
None\n\n                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       17358
\n\n                                                                                      